In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib.request

In [2]:
def get_champ_win_rate(name):
    
    result = requests.get(f"https://na.op.gg/champion/{name}/statistics/top/matchup",headers={'User-Agent': 'Mozilla/5.0'})
    src = result.content
    soup = BeautifulSoup(src,'lxml')
    
    #list of all span tags
    spans = soup.find_all("span")
    
    #converting above list to a list of strings
    tags = []
    for tag in spans:
        tags.append(str(tag))
        
    #the win rate figures are only on lines with this string
    win_rate = []
    for tag in tags:
        if "champion-matchup-list__winrate" in tag:
            win_rate.append(tag)
            
    #the champion name is directly above the win rate figure line. so we are finding the win rate figure in tags, and then 
    #grabbing the above champion name in the tags list
    champions = []
    for win in win_rate:
        champions.append(tags[tags.index(win) - 1])
    
    #cleaning up the line to just have the champion name
    champions2 = []
    for champ in champions:
        champ = champ.replace('<span>','')
        champ = champ.replace('</span>','')
        champions2.append(champ)
    
    #cleaning up the line to just have the win rate
    win_rate2 = []
    for win in win_rate:
        win = win.replace('<span class="champion-matchup-list__winrate">','')
        win = win.replace('\n','')
        win = win.replace('\t','')
        win = win.replace('</span>','')
        win = win.replace('%','')
        win_rate2.append(win)
        
    df = pd.DataFrame(win_rate2,champions2).reset_index()
    
    df = df.rename(columns={'index':'Champs',0:f"{name}_Win_Rate"})
    df[f"{name}_Win_Rate"] = df[f"{name}_Win_Rate"].astype(float)/100
    
    return df

def counter_pick(opponent,champ_pool=['Darius','Garen','Dr. Mundo','Mordekaiser','Fiora','Trundle']):
    
    #removing opponent if the opponent is in the champ_pool
    if opponent in champ_pool:
        champ_pool.remove(opponent)
    
    #creating a list of dataframes. each df is the win rates for all opponents with enough data
    champ_df_list = []
    for champ in champ_pool:
        champ = get_champ_win_rate(champ)
        champ_df_list.append(champ)
    
    #if an opponent doesn't appear in one of our df's because it's such an obscure opponent, we remove that df
    for y in champ_df_list:
        if y.loc[y.Champs==opponent].shape[0] == 0:
            champ_df_list.remove(y)
    
    #creating one master dataframe of all the indivdual champ df's. 
    started = False        
    for x in champ_df_list:
        if not started:
            df = x
            started = True
        else:
            df = df.merge(x,how='inner',on='Champs')
            
    df = df.drop_duplicates()        
    df = df.loc[df.Champs==opponent]
    
    return(df.set_index('Champs').transpose().sort_values(by=opponent,ascending=False))
        

In [3]:
counter_pick('Riven',champ_pool=['Darius','Garen','Dr. Mundo','Mordekaiser','Trundle','Pantheon'])

Champs,Riven
Garen_Win_Rate,0.5217
Pantheon_Win_Rate,0.4889
Trundle_Win_Rate,0.4833
Darius_Win_Rate,0.4777
Mordekaiser_Win_Rate,0.4699
Dr. Mundo_Win_Rate,0.4587


In [7]:
x = counter_pick('Gnar',champ_pool=['Darius','Garen','Dr. Mundo','Mordekaiser','Fiora','Trundle'])